

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/TEXT_PREPROCESSING.ipynb)




# **Pre-Process text:**
## **Convert text to tokens, remove punctuation, stop words, perform stemming and lemmatization using Spark NLP's annotators**

**Demo of the following annotators:**


* SentenceDetector
* Tokenizer
* Normalizer
* Stemmer
* Lemmatizer
* StopWordsCleaner

## 1. Colab Setup

In [1]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash
# !bash colab.sh
# -p is for pyspark
# -s is for spark-nlp
# !bash colab.sh -p 3.1.1 -s 3.0.1
# by default they are set to the latest

openjdk version "11.0.10" 2021-01-19
OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
setup Colab for PySpark 3.1.1 and Spark NLP 3.0.0
     |████████████████████████████████| 212.3MB 60kB/s 
     |████████████████████████████████| 143kB 42.0MB/s 
     |████████████████████████████████| 204kB 40.4MB/s 


In [2]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

## 2. Start Spark Session

In [3]:
spark = sparknlp.start()

## 3. Setting sample text

In [4]:
## Generating Example Files ##

text_list = ["""The Geneva Motor Show, the first major car show of the year, opens tomorrow with U.S. Car makers hoping to make new inroads into European markets due to the cheap dollar, automobile executives said. Ford Motor Co and General Motors Corp sell cars in Europe, where about 10.5 mln new cars a year are bought. GM also makes a few thousand in North American plants for European export.""",
             ]

## 4. Download lemma reference file. (you may also use a pre-trained lemmatization model)

In [5]:
#getting lemma files
!wget https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt

--2021-04-04 11:27:25--  https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1348552 (1.3M) [text/plain]
Saving to: ‘AntBNC_lemmas_ver_001.txt’

AntBNC_lemmas_ver_0 100%[===================>]   1.29M  --.-KB/s    in 0.02s   

2021-04-04 11:27:25 (82.3 MB/s) - ‘AntBNC_lemmas_ver_001.txt’ saved [1348552/1348552]



## 5. Define Spark NLP pipleline

In [6]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentences')

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")\
    .setLowercase(True)\
    .setCleanupPatterns(["[^\w\d\s]"])

stopwords_cleaner = StopWordsCleaner()\
    .setInputCols("token")\
    .setOutputCol("removed_stopwords")\
    .setCaseSensitive(False)\

stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")


lemmatizer = Lemmatizer() \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \
    .setDictionary("./AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

nlpPipeline = Pipeline(stages=[documentAssembler,
                               sentenceDetector,
                               tokenizer,
                               normalizer,
                               stopwords_cleaner,
                               stemmer,
                               lemmatizer,
                               ])


## 6. Run pipeline

In [7]:
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

df = spark.createDataFrame(pd.DataFrame({'text':text_list}))
result = pipelineModel.transform(df)

## 7. Visualize Results

In [8]:
# sentences in the text
result.select(F.explode(F.arrays_zip('sentences.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("sentences")).show(truncate=False)


+----------------------------------------------------------------------------------------------------------------+
|sentences                                                                                                       |
+----------------------------------------------------------------------------------------------------------------+
|The Geneva Motor Show, the first major car show of the year, opens tomorrow with U.S.                           |
|Car makers hoping to make new inroads into European markets due to the cheap dollar, automobile executives said.|
|Ford Motor Co and General Motors Corp sell cars in Europe, where about 10.5 mln new cars a year are bought.     |
|GM also makes a few thousand in North American plants for European export.                                      |
+----------------------------------------------------------------------------------------------------------------+



In [9]:
# tokens in the text
result.select(F.explode(F.arrays_zip('token.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("tokens")).show(truncate=False)

+--------+
|tokens  |
+--------+
|The     |
|Geneva  |
|Motor   |
|Show    |
|,       |
|the     |
|first   |
|major   |
|car     |
|show    |
|of      |
|the     |
|year    |
|,       |
|opens   |
|tomorrow|
|with    |
|U.S     |
|.       |
|Car     |
+--------+
only showing top 20 rows



In [10]:
# eliminated punctuation
result.select(F.explode(F.arrays_zip('normalized.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("normalized_tokens")).show(truncate=False)

+-----------------+
|normalized_tokens|
+-----------------+
|the              |
|geneva           |
|motor            |
|show             |
|the              |
|first            |
|major            |
|car              |
|show             |
|of               |
|the              |
|year             |
|opens            |
|tomorrow         |
|with             |
|us               |
|car              |
|makers           |
|hoping           |
|to               |
+-----------------+
only showing top 20 rows



In [11]:
# stemmed tokens
result.select(F.explode(F.arrays_zip('stem.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token_stems")).show(truncate=False)

+-----------+
|token_stems|
+-----------+
|the        |
|geneva     |
|motor      |
|show       |
|,          |
|the        |
|first      |
|major      |
|car        |
|show       |
|of         |
|the        |
|year       |
|,          |
|open       |
|tomorrow   |
|with       |
|u.         |
|.          |
|car        |
+-----------+
only showing top 20 rows



In [12]:
# removed_stopwords
result.select(F.explode(F.arrays_zip('removed_stopwords.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("removed_stopwords")).show(truncate=False)

+-----------------+
|removed_stopwords|
+-----------------+
|Geneva           |
|Motor            |
|Show             |
|,                |
|first            |
|major            |
|car              |
|show             |
|year             |
|,                |
|opens            |
|tomorrow         |
|U.S              |
|.                |
|Car              |
|makers           |
|hoping           |
|make             |
|new              |
|inroads          |
+-----------------+
only showing top 20 rows



In [13]:
# lemmatization
result.select(F.explode(F.arrays_zip('lemma.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("lemma")).show(truncate=False)

+--------+
|lemma   |
+--------+
|The     |
|Geneva  |
|Motor   |
|Show    |
|,       |
|the     |
|first   |
|major   |
|car     |
|show    |
|of      |
|the     |
|year    |
|,       |
|open    |
|tomorrow|
|with    |
|U.S     |
|.       |
|Car     |
+--------+
only showing top 20 rows

